# Problem logarytmu dyskretnego

Niech $(G,\circ)$ będzie grupą z działaniem $\circ$ i elementem neutralnym $1_G$. Wtedy dla dowolnego elementu $a\in G$ i $k\in\mathbb{Z}$ definiujemy *potęgę* $$a^k =\left\{\begin{array}{cc}
\underbrace{a\circ a\circ \ldots \circ a}_{k}&\text{ dla }k>0\\
1_G&\text{ dla }k=0\\
\underbrace{a^{-1}\circ a^{-1}\circ \ldots \circ a^{-1}}_{k}&\text{ dla }k<0
\end{array}\right.$$
gdzie $a^{-1}$ jest elementem odwrotnym do $a$.

Dla $a,b\in G$, $b\neq 1_G$, **logarytmem dyskretnym** $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a$.

## Logarytm dyskretny w $\mathbb{Z}_n$

W przypadku pierścienia $\mathbb{Z}_n$ logarytmem dyskretnym $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a\mod n$, o ile w ogóle istnieje.

Specyficzną sytuacją w teorii liczb jest gdy $n=p$ jest liczbą pierwszą a $q$ jest **pierwiastkiem pierwotnym** $\mod p$. Wtedy:
- potęgi $q^k\mod p$ generują cały zbiór $[1,p-1]$, tzn. $q$ jest **generatorem grupy multiplikatywnej** rzędu $p-1$
- logarytm dyskretny $\log_q a$ istnieje dla każdego niezerowego elementu $a\in \mathbb{Z}_p$


# Wymiana klucza typu Diffie-Hellman

Alice i Bob uzgadniają klucz publiczny będący liczbą pierwszą $p$ oraz $q$ - pierwiastkiem pierwotnym mod $p$.
- sekret Alice: liczba całkowita $n\in \mathbb{Z}_p\setminus\{0\}$
- sekret Boba: liczba całkowita $m\in \mathbb{Z}_p\setminus\{0\}$
- Alice generuje $x=q^n\mod p$ i wysyła do Boba
- Bob generuje $y=q^m\mod p$ i wysyła Alice
- Alice oblicza klucz $k=y^n\mod p$
- Bob oblicza klucz $k=x^m\mod p$


## Zadanie 1.

Zaimplementuj powyższy algorytm wymiany klucza. Dobierz parametry $p$ i $q$ tak, żeby znając $x$, $y$, $p$ i $q$ nie dało się odtworzyć sekretów algorytmem z zadania 1.

In [4]:
import random

p = 2089 
q = 2 

n = random.randint(1, p-1)
m = random.randint(1, p-1)

x = pow(q, n, p) 
y = pow(q, m, p)

k_alice = pow(y, n, p) 
k_bob   = pow(x, m, p) 


print("pub: \n")
print("p =", p, "q =", q)
print("\npriv: \n")
print("n (Alice) =", n)
print("m (Bob)   =", m)
print("\nshared: \n")
print("k_alice =", k_alice)
print("k_bob   =", k_bob)
print("\nDo keys equal? ", k_alice == k_bob)


pub: 

p = 2089 q = 2

priv: 

n (Alice) = 950
m (Bob)   = 1173

shared: 

k_alice = 914
k_bob   = 914

Do keys equal?  True


## Algorytm baby-step giant-step

Jeden z najprostszych (poza metodą naiwną) algorytmów poszukiwania logarytmu dyskretnego w grupach cyklicznych.

Niech $p$ będzie liczbą pierwszą oraz niech $q$ będzie pierwiastkiem pierwotnym modulo $p$. Dla niezerowego $a\in\mathbb{Z}_p$ szukamy liczby $k\in\mathbb{Z}$ takiej, że $q^k=a\mod p$

### Krok 1.
- $m=\lceil\sqrt{p-1}\rceil$
- tworzymy pomocniczą tablicę potęg: dla wszystkich $i\in [0,m)$ obliczamy parę $(i,q^i)$
- obliczamy $r=(q^{-1})^m$
### Krok 2.
- $b=a$
- dla wszystkich $j\in [0,m)$:
    - sprawdzamy, czy para $(i,b)$ jest elementem tablicy potęg dla pewnego $i$
    - jeżeli tak, to $k=jm+i$ i kończymy algorytm
    - jeżeli nie, to $b=br$ i kontynuujemy pętlę

## Zadanie 2.

Zaimplementować algorytm baby-step giant-step. Przetestować dla podanych danych testujących.

```Dane testujące:
p = 7
q = 3
a = 4

m = 3
tablica_testowa = [1,3,2]
r = 6
k = 4 (j = 1, i = 1)
```

```
p = 29
q = 8
a = 10

m = 6
tablica_testowa = [1,8,6,19,7,27]
r = 9
k = 17 (j = 2, i = 5)
```

```
p = 113
q = 76
a = 84

m = 11
tablica_testowa = [1,76,13,84,56,75,50,71,85,19,88]
r = 70
k = 3 (j = 0, i = 3)
```

In [6]:
import math

def baby_step_giant_step(p, q, a):
    m = math.isqrt(p-1) + 1

    baby_steps = {}
    value = 1
    for i in range(m):
        baby_steps[value] = i 
        value = (value * q) % p

    q_inv = pow(q, -1, p) 
    r = pow(q_inv, m, p)

    b = a
    for j in range(m):
        if b in baby_steps:
            i = baby_steps[b]
            k = j * m + i
            return k
        b = (b * r) % p
    return None 

In [12]:
testy = [
    (7, 3, 4, 4),
    (29, 8, 10, 17),
    (113, 76, 84, 3)
]

for p, q, a, expected in testy:
    k = baby_step_giant_step(p, q, a)
    print(f"p={p}, q={q}, a={a} -> k={k}, expected={expected}, correct={k==expected}")

p=7, q=3, a=4 -> k=4, expected=4, correct=True
p=29, q=8, a=10 -> k=17, expected=17, correct=True
p=113, q=76, a=84 -> k=3, expected=3, correct=True
